In [1]:

import pandas as pd
import glob



/Users/shiyiwang/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/shiyiwang/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
def extract_company_name(filename):
    return filename.split("ESG_Extracted-")[-1].split("_")[0]

all_rows = []

for file in glob.glob("ESG_Extracted-*.xlsx"):
    company = extract_company_name(file)
    row = {"Company": company, "G_value": 0}
    
    # 处理E Sheet
    try:
        e_df = pd.read_excel(file, sheet_name="E", usecols=["Keywords", "Value"])
        e_dict = e_df.groupby("Keywords")["Value"].last().to_dict()
        row.update(e_dict)
    except:
        pass
    
    # 处理S Sheet
    try:
        s_df = pd.read_excel(file, sheet_name="S", usecols=["Keywords", "Value"])
        s_dict = s_df.groupby("Keywords")["Value"].last().to_dict()
        row.update(s_dict)
    except:
        pass
    
    # 处理G Sheet
    try:
        g_df = pd.read_excel(file, sheet_name="G", usecols=["Value"])
        row["G_value"] = g_df["Value"].sum()
    except:
        pass
    
    all_rows.append(row)

# 合并所有数据并填充缺失值为0
final_df = pd.DataFrame(all_rows).fillna(0)

# 按字母顺序排序列（公司名和G_value固定在前）
cols = ["Company"] + [col for col in sorted(final_df.columns) if col not in ["Company", "G_value"]] + ["G_value"]
final_df = final_df[cols]

final_df.to_csv("consolidated_esg_single_row.csv", index=False)